
# Part A – Data Preparation

This notebook covers:

• Loading both datasets  
• Documenting rows/columns  
• Checking missing values and duplicates  
• Converting timestamps  
• Aligning datasets by daily date  
• Creating key trading metrics:
    - Daily PnL per trader
    - Win rate
    - Average trade size
    - Leverage distribution
    - Trades per day
    - Long/Short ratio


In [2]:

import pandas as pd
import numpy as np

# Load datasets
trades = pd.read_csv('historical_data.csv')
sentiment = pd.read_csv('fear_greed_index.csv')

# Standardize column names
trades.columns = trades.columns.str.lower().str.strip()
sentiment.columns = sentiment.columns.str.lower().str.strip()

print("Trades shape:", trades.shape)
print("Sentiment shape:", sentiment.shape)


Trades shape: (211224, 16)
Sentiment shape: (2644, 4)


In [3]:

print("\nMissing values (Trades):\n", trades.isnull().sum())
print("\nMissing values (Sentiment):\n", sentiment.isnull().sum())

print("\nDuplicate rows (Trades):", trades.duplicated().sum())
print("Duplicate rows (Sentiment):", sentiment.duplicated().sum())



Missing values (Trades):
 account             0
coin                0
execution price     0
size tokens         0
size usd            0
side                0
timestamp ist       0
start position      0
direction           0
closed pnl          0
transaction hash    0
order id            0
crossed             0
fee                 0
trade id            0
timestamp           0
dtype: int64

Missing values (Sentiment):
 timestamp         0
value             0
classification    0
date              0
dtype: int64

Duplicate rows (Trades): 0
Duplicate rows (Sentiment): 0


In [4]:

# Detect date/time columns automatically
trade_date_col = [c for c in trades.columns if 'date' in c or 'time' in c][0]
sent_date_col = [c for c in sentiment.columns if 'date' in c or 'time' in c][0]

# Convert to daily date
trades['date'] = pd.to_datetime(trades[trade_date_col], errors='coerce').dt.date
sentiment['date'] = pd.to_datetime(sentiment[sent_date_col], errors='coerce').dt.date

# Merge datasets
data = pd.merge(trades, sentiment[['date']], on='date', how='left')

print("Merged dataset shape:", data.shape)


Merged dataset shape: (211224, 17)


In [5]:

# Detect numeric columns
numeric_cols = trades.select_dtypes(include=np.number).columns.tolist()

# Assign PnL column (first numeric column assumed)
pnl_col = numeric_cols[0]
trades.rename(columns={pnl_col: 'pnl'}, inplace=True)

# Create Win flag
trades['win'] = np.where(trades['pnl'] > 0, 1, 0)

# Daily PnL per trader (if trader column exists)
trader_col = [c for c in trades.columns if 'trader' in c or 'account' in c or 'user' in c]

if trader_col:
    daily_pnl = trades.groupby(['date', trader_col[0]])['pnl'].sum().reset_index()
else:
    daily_pnl = trades.groupby(['date'])['pnl'].sum().reset_index()

# Win rate
win_rate = trades['win'].mean()

# Average trade size (if exists)
if len(numeric_cols) > 1:
    trades.rename(columns={numeric_cols[1]: 'trade_size'}, inplace=True)
    avg_trade_size = trades['trade_size'].mean()
else:
    avg_trade_size = None

# Leverage distribution (if exists)
if len(numeric_cols) > 2:
    trades.rename(columns={numeric_cols[2]: 'leverage'}, inplace=True)
    leverage_dist = trades['leverage'].describe()
else:
    leverage_dist = None

# Trades per day
trades_per_day = trades.groupby('date').size()

# Long/Short ratio (if side column exists)
side_col = [c for c in trades.columns if 'side' in c]
if side_col:
    long_short = trades.groupby(['date', side_col[0]]).size().unstack().fillna(0)
else:
    long_short = None

print("Win Rate:", win_rate)
print("Average Trade Size:", avg_trade_size)
print("Leverage Distribution:\n", leverage_dist)
print("Trades per Day sample:\n", trades_per_day.head())


Win Rate: 1.0
Average Trade Size: 4623.364978782207
Leverage Distribution:
 count    2.112240e+05
mean     5.639451e+03
std      3.657514e+04
min      0.000000e+00
25%      1.937900e+02
50%      5.970450e+02
75%      2.058960e+03
max      3.921431e+06
Name: leverage, dtype: float64
Trades per Day sample:
 date
2023-01-05      3
2023-05-12      9
2024-01-01     18
2024-01-02      6
2024-01-03    137
dtype: int64
